In [1]:
import pandas as pd
import os
from datetime import date, datetime
import numpy as np

In [2]:
dirpath = os.getcwd()
data_path = os.path.join(dirpath, "..", "data")
df = pd.read_csv(os.path.join(data_path, "merged_lactation.csv"))


## find the teats that did not produce milk

In [3]:
list_temp = []
thres = 5
for cow in df.animal_number:
    cow_line = df[df.animal_number == cow]
    lfy = cow_line["yield_lf_visit_sum"].values[0]
    rfy = cow_line["yield_rf_visit_sum"].values[0]
    lby = cow_line["yield_lr_visit_sum"].values[0]
    rby = cow_line["yield_rr_visit_sum"].values[0]
    if lfy < thres:
        list_temp.append([cow, 'lf', lfy])
    if rfy < thres:
        list_temp.append([cow, 'rf', rfy])
    if lby < thres:
        list_temp.append([cow, 'rr', lby])
    if rby < thres:
        list_temp.append([cow, 'lr',rby])

missing_teat_df = pd.DataFrame(list_temp, columns = ["cow", "teat", 'y'])
missing_teat_df.to_csv(os.path.join(data_path,"missing_teats.csv"), index = False)

## change values from missing teats to NA

In [4]:
df2 = df.copy()
cows_missing = np.unique(missing_teat_df.cow)

for cow in cows_missing:
    teats_missing = missing_teat_df[missing_teat_df.cow == cow].reset_index()["teat"]
    for teat in teats_missing:
        columns_na = [col for col in df.columns if teat in col]
        idx = np.where(df2.animal_number == cow)[0]
        df2.loc[idx, columns_na] = np.nan

df2.to_csv(os.path.join(data_path,"merged2_lactation.csv"), index = False)

In [5]:
df2.columns

Index(['animal_number', 'min_date', 'max_date', 'count_date', 'min_dim',
       'max_dim', 'milk_flow_duration_mean', 'yield_visit_mean',
       'yield_lf_visit_mean', 'yield_lr_visit_mean', 'yield_rr_visit_mean',
       'yield_rf_visit_mean', 'peak_flow_rf_mean', 'peak_flow_lf_mean',
       'peak_flow_rr_mean', 'peak_flow_lr_mean', 'notmilk_visit_duration_mean',
       'kickoff_lf_sum', 'kickoff_lr_sum', 'kickoff_rf_sum', 'kickoff_rr_sum',
       'kickoff_front_sum', 'kickoff_back_sum', 'kickoff_any_sum',
       'total_yield', 'total_visits', 'kickoff_lf_perc', 'kickoff_lr_perc',
       'kickoff_rf_perc', 'kickoff_rr_perc', 'kickoff_front_perc',
       'kickoff_back_perc', 'kickoff_any_perc', 'yield_lf_visit_sum',
       'yield_lr_visit_sum', 'yield_rr_visit_sum', 'yield_rf_visit_sum',
       'interval_visit_mean'],
      dtype='object')